# Uber Analysis #

**To:** Operations & Strategy Stakeholders (Uber Rides Division)

**From:** Jean Fernandes (Data Analytics Team)

**Subject:** Analysis of Ride Data for Operational Insights - Draft



## Executive Summary

We analyzed the dataset...

## Loading the data

In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("yashdevladdha/uber-ride-analytics-dashboard")

print("Path to dataset files:", path)

ModuleNotFoundError: No module named 'kagglehub'